In [1]:
###########################################################################
#  Code for computing the RPS and IR scores for a given evaluation period
###########################################################################

#For simplicity, in this example it is assumed that the data provided cover a single evaluation period.
#This period is specified through the min/max date of the asset prices data set.
#If you wish to compute RPS/IR for multiple periods, you'll have to execute 
#the script multiple times, each time using a different, appropriate input.

import pandas as pd
import numpy as np
from statistics import stdev

In [37]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [2]:
data_in     = "../DataRaw/"
data_folder = "../DataWork/"

In [49]:
#Read asset prices data (as provided by the M6 submission platform)
asset_data = pd.read_csv(data_in + "assets_m6.csv")

#Read submission file (similar to the template provided by the M6 submission platform)
submission_data = pd.read_csv(data_folder + "submit_trial_2022_02_05.csv")

hist_data = asset_data
submission = submission_data

In [50]:
hist_data

,symbol,date,price
0,ABBV,2022/01/31,136.890
1,ACN,2022/01/31,353.580
2,AEP,2022/01/31,89.618
3,AIZ,2022/01/31,151.876
4,ALLE,2022/01/31,122.730
...,...,...,...
2705,XLP,2022/03/09,72.970
2706,XLU,2022/03/09,70.200
2707,XLV,2022/03/09,129.960
2708,XLY,2022/03/09,168.790


In [51]:
#hist_data[hist_data.symbol == 'IEFM.L']

In [52]:
hist_data = hist_data[hist_data.date > "2022/03/03"]
hist_data

,symbol,date,price
2310,ABBV,2022/03/04,150.560
2311,ACN,2022/03/04,314.160
2312,AEP,2022/03/04,96.330
2313,AIZ,2022/03/04,169.390
2314,ALLE,2022/03/04,117.290
2315,AMAT,2022/03/04,125.740
2316,AMP,2022/03/04,281.490
2317,AMZN,2022/03/04,2912.820
2318,AVB,2022/03/04,247.340
2319,AVY,2022/03/04,167.590


In [79]:
#Function for computing RPS
def RPS_calculation(hist_data, submission, asset_no=100):
    
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                              'symbol' : asset,
                                              'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price  = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0],
                                  'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    #Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:
        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : ranking}
    
    return(output)
   
#Function for computing IR
def IR_calculation(hist_data, submission):
    
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                              'symbol' : asset,
                                              'price' : right_price}, ignore_index=True)

    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])

    #Investment weights
    weights = submission[["ID","Decision"]]

    RET = pd.DataFrame()

    for i in range(len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        temp = temp.sort_values(by='date')
        temp.reset_index(inplace=True, drop=True)
        RET = RET.append(temp.price.pct_change()*weights.loc[weights.ID==asset_id[i]].Decision.values[0])

    ret = np.log(1+RET.sum()[1:])
    sum_ret = sum(ret)
    sdp = stdev(ret)
    
    output = {'IR' : sum_ret/sdp,
              'details' : RET} #list(ret)}
    
    return output

In [115]:
w = submission[['ID','Decision']].set_index('ID').T
w

ID,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,BDX,BF-B,BMY,BR,CARR,CDW,CE,CHTR,CNC,CNP,COP,CTAS,CZR,DG,DPZ,DRE,DXC,FB,FTV,GOOG,GPC,HIG,HST,JPM,KR,OGN,PG,PPL,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,WRK,XOM,IVV,IWM,EWU,EWG,EWL,EWQ,IEUS,EWJ,EWT,MCHI,INDA,EWY,EWA,EWH,EWZ,EWC,IEMG,LQD,HYG,SHY,IEF,TLT,SEGA.L,IEAA.L,HIGH.L,JPEA.L,IAU,SLV,GSG,REET,ICLN,IXN,IGF,IUVL.L,IUMO.L,SPMV.L,IEVL.L,IEFM.L,MVEU.L,XLK,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
Decision,0.01,-0.02,0.0,0.0,-0.01,-0.02,0.0,-0.01,0.0,-0.01,0.01,0.01,-0.01,0.01,-0.02,-0.02,-0.01,-0.01,-0.01,0.0,0.0,0.03,-0.02,-0.02,-0.02,-0.03,-0.01,0.01,-0.04,-0.02,0.0,-0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,-0.04,0.0,-0.01,-0.02,0.0,-0.01,0.01,-0.02,0.0,0.03,-0.01,-0.01,0.0,0.0,-0.01,0.0,-0.01,0.0,0.0,0.0,0.0,-0.01,-0.01,0.0,0.01,0.0,0.0,-0.01,0.0,0.0,0.0,-0.01,-0.02,0.0,0.0,0.0,0.0,0.0,0.02,-0.01,-0.02,-0.01,0.0,0.0,-0.01,-0.01,0.0,0.11,-0.01,-0.01,0.0,-0.01,0.02,-0.01,-0.01,-0.01,0.0,0.0,-0.01,0.05


In [127]:
t = hist_data.set_index('date').pivot(columns='symbol').droplevel(0, axis=1).pct_change() * w[t.columns].values
t

symbol,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,BDX,BF-B,BMY,BR,CARR,CDW,CE,CHTR,CNC,CNP,COP,CTAS,CZR,DG,DPZ,DRE,DXC,EWA,EWC,EWG,EWH,EWJ,EWL,EWQ,EWT,EWU,EWY,EWZ,FB,FTV,GOOG,GPC,GSG,HIG,HIGH.L,HST,HYG,IAU,ICLN,IEAA.L,IEF,IEFM.L,IEMG,IEUS,IEVL.L,IGF,INDA,IUMO.L,IUVL.L,IVV,IWM,IXN,JPEA.L,JPM,KR,LQD,MCHI,MVEU.L,OGN,PG,PPL,PRU,PYPL,RE,REET,ROL,ROST,SEGA.L,SHY,SLV,SPMV.L,TLT,UNH,URI,V,VRSK,VXX,WRK,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOM
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022/03/04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022/03/07,-0.000074,0.000582,0.0,-0.0,0.000218,0.001037,-0.0,0.000562,-0.0,0.000538,-0.000799,0.000023,0.000292,-0.000085,0.000022,0.000658,0.000357,0.000584,-0.000104,-0.0,-0.0,0.000299,0.000419,0.002325,0.000688,0.000989,0.000071,-0.000350,0.000194,-0.0,-0.0,-0.0,-0.0,0.000372,-0.0,-0.0,-0.0,0.000411,-0.000367,0.002517,0.000931,-0.0,0.000326,0.000536,-0.0,-0.0,-0.0,-0.0,0.0,-0.000494,-0.0,-0.0,-0.001041,-0.0,0.000411,-0.0,-0.0,-0.0,0.000128,-0.0,0.000294,0.000249,0.000391,-0.0,-0.0,-0.0,0.000121,-0.0,0.000069,-0.000103,-0.0,-0.0,-0.000412,0.002522,-0.0,0.000205,0.000015,0.000966,0.000060,-0.0,0.0,0.000029,0.000076,-0.0,0.000577,-0.000479,0.000021,0.004370,-0.0,0.000347,0.000332,0.000295,-0.0,0.000268,0.000366,-0.0,0.0,0.000174,0.000490,0.001081
2022/03/08,-0.000154,0.000223,-0.0,-0.0,0.000288,-0.000827,-0.0,0.000105,-0.0,-0.000021,0.000058,-0.000396,0.000548,-0.000150,0.000299,-0.000028,0.000040,-0.000021,0.000076,-0.0,-0.0,-0.000732,0.000189,-0.000172,0.000438,0.000519,0.000160,0.000660,0.000156,-0.0,0.0,0.0,-0.0,-0.000047,0.0,0.0,0.0,-0.000070,0.000064,-0.000602,0.000516,0.0,-0.000099,0.000755,0.0,-0.0,0.0,-0.0,0.0,-0.001377,-0.0,-0.0,-0.000845,0.0,-0.000367,0.0,0.0,0.0,0.000280,-0.0,0.000075,-0.000055,0.000006,-0.0,-0.0,-0.0,0.000060,-0.0,0.000201,-0.000461,-0.0,0.0,0.000162,-0.000551,0.0,0.000011,0.000409,-0.000174,0.000041,-0.0,0.0,0.000212,0.000101,-0.0,-0.000417,0.000053,0.000263,-0.000597,-0.0,0.000051,0.000025,0.000314,-0.0,0.000061,0.000049,-0.0,-0.0,0.000204,-0.000021,0.000227
2022/03/09,0.000115,-0.000517,-0.0,0.0,-0.000279,-0.000720,0.0,-0.000240,0.0,-0.000433,0.000537,-0.000011,-0.000030,0.000058,-0.000274,-0.000464,-0.000377,-0.000352,-0.000115,0.0,-0.0,-0.000786,-0.000362,-0.002093,-0.000799,-0.000308,-0.000134,-0.000285,-0.000213,0.0,0.0,-0.0,0.0,-0.000448,0.0,0.0,0.0,-0.000395,0.000271,-0.001726,-0.000782,0.0,-0.000146,-0.002143,0.0,0.0,0.0,0.0,-0.0,-0.000056,0.0,-0.0,0.006120,0.0,-0.000494,0.0,0.0,0.0,-0.000341,0.0,-0.000269,-0.000282,-0.000413,0.0,0.0,-0.0,-0.000017,0.0,-0.000385,-0.000052,0.0,0.0,0.000308,-0.002242,0.0,-0.000192,-0.000046,-0.000631,-0.000021,-0.0,-0.0,-0.000116,0.000098,0.0,-0.000164,0.000420,-0.000242,-0.001989,0.0,-0.000310,-0.000299,-0.000611,0.0,-0.000181,-0.000396,0.0,-0.0,-0.000189,-0.000322,-0.001705


In [136]:
t.sum().sort_values()

symbol
ICLN     -0.001928
COP      -0.001219
GSG      -0.000853
OGN      -0.000616
AMAT     -0.000510
IEUS     -0.000449
XOM      -0.000397
BDX      -0.000384
PYPL     -0.000271
AXP      -0.000204
BMY      -0.000177
CHTR     -0.000143
EWL      -0.000123
MVEU.L   -0.000115
ABBV     -0.000113
IWM      -0.000088
EWY      -0.000054
EWZ      -0.000033
IXN      -0.000016
V        -0.000006
URI      -0.000004
XLE      -0.000002
IEF       0.000000
AMP       0.000000
AEP       0.000000
IEVL.L    0.000000
IGF       0.000000
INDA      0.000000
IUVL.L    0.000000
XLU       0.000000
IEAA.L    0.000000
WRK       0.000000
JPEA.L    0.000000
KR        0.000000
MCHI      0.000000
IAU       0.000000
PG        0.000000
PPL       0.000000
XLP       0.000000
RE        0.000000
SHY       0.000000
SLV       0.000000
UNH       0.000000
XLF       0.000000
JPM       0.000000
HYG       0.000000
IEMG      0.000000
HIGH.L    0.000000
HIG       0.000000
AVB       0.000000
AIZ       0.000000
CNP       0.000000
GOOG 

In [129]:
rets = np.log(1+t.sum(axis=1)[1:])
rets

date
2022/03/07    0.024207
2022/03/08   -0.000286
2022/03/09   -0.018646
dtype: float64

In [134]:
sum(rets)/stdev(rets)

0.24535732866399415

In [71]:
t =  IR_calculation(hist_data = hist_data , submission = submission_data)
t['details']

,0,1,2,3
price,NaN,-0.000074,-0.000154,0.000115
price,NaN,0.000582,0.000223,-0.000517
price,NaN,0.000000,-0.000000,-0.000000
price,NaN,-0.000000,-0.000000,0.000000
price,NaN,0.000218,0.000288,-0.000279
price,NaN,0.001037,-0.000827,-0.000720
price,NaN,-0.000000,-0.000000,0.000000
price,NaN,0.000562,0.000105,-0.000240
price,NaN,-0.000000,-0.000000,0.000000
price,NaN,0.000538,-0.000021,-0.000433


In [78]:
np.log(1+t['details'].sum()[1:])

1    0.024207
2   -0.000286
3   -0.018646
dtype: float64

In [54]:
#Run evaluation
rps_dict = RPS_calculation(hist_data = hist_data , submission = submission_data)
rps = rps_dict['RPS']
ir_dict = IR_calculation(hist_data, submission)
ir = ir_dict['IR']

print(f'RPS {rps:.3f} IR {ir:.3f}')

RPS 0.324 IR 0.245


In [55]:
hist_data.date.unique()

array(['2022/03/04', '2022/03/07', '2022/03/08', '2022/03/09'],
      dtype=object)

In [56]:
ir_dict['details']

[0.02420739751824224, -0.00028644861498379297, -0.018645845814299303]

In [57]:
rps_dict['details'].sort_values(by='ID').reset_index(drop=True)

,ID,Return,Position,Rank,Rank1,Rank2,Rank3,Rank4,Rank5
0,ABBV,-0.011424,43.0,3.0,0.0,0.0,1.0,0.0,0.0
1,ACN,-0.015088,36.0,2.0,0.0,1.0,0.0,0.0,0.0
2,AEP,-0.013703,38.0,2.0,0.0,1.0,0.0,0.0,0.0
3,AIZ,-0.003660,62.0,4.0,0.0,0.0,0.0,1.0,0.0
4,ALLE,-0.023446,22.0,2.0,0.0,1.0,0.0,0.0,0.0
5,AMAT,0.022904,90.0,5.0,0.0,0.0,0.0,0.0,1.0
6,AMP,-0.024761,20.0,1.0,1.0,0.0,0.0,0.0,0.0
7,AMZN,-0.043683,5.0,1.0,1.0,0.0,0.0,0.0,0.0
8,AVB,-0.011644,42.0,3.0,0.0,0.0,1.0,0.0,0.0
9,AVY,-0.010681,45.0,3.0,0.0,0.0,1.0,0.0,0.0


In [58]:
submission.sort_values(by='ID')

,ID,Rank1,Rank2,Rank3,Rank4,Rank5,Decision,RPS
0,ABBV,0,0,0,0,1,0.01,0.4
1,ACN,1,0,0,0,0,-0.02,0.2
2,AEP,0,0,0,1,0,0.00,0.4
3,AIZ,0,0,0,1,0,0.00,0.0
4,ALLE,1,0,0,0,0,-0.01,0.2
5,AMAT,1,0,0,0,0,-0.02,0.8
6,AMP,0,0,0,1,0,0.00,0.6
7,AMZN,0,0,1,0,0,-0.01,0.4
8,AVB,0,0,0,1,0,0.00,0.2
9,AVY,1,0,0,0,0,-0.01,0.4
